In [39]:
with open('processed_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chunk = 1000
chunks = [text[i: i + chunk] for i in range(0, len(text), chunk)]

In [40]:
from sentence_transformers import SentenceTransformer  # SentenceTransformer.shape() --> مثلاً ۳۸۴ یا ۷۶۸ بعدی
import numpy as np

# مدل چندزبانه MiniLM که فارسی رو هم پشتیبانی می‌کنه
embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# گرفتن embedding برای هر chunk
embeddings = embedder.encode(chunks, convert_to_tensor=False)
embeddings = np.array(embeddings).astype("float32")  # خروجی تبدیل به float32 میشه چون FAISS فقط با float32 کار می‌کنه.

# اینجا embeddingها رو داخل یک دیتابیس سریع جستجو قرار می‌دیم:
import faiss

dimension = embeddings.shape[1]  # تعداد ابعاد بردار (مثلاً 384)
index = faiss.IndexFlatL2(dimension)  # یعنی جستجو بر اساس فاصله اقلیدسی L2
index.add(embeddings)


In [41]:
# جستجوی chunkهای مرتبط (فقط متن‌ها)
def retrive_relevant_chunks(query, top_k=3):
    query_embedding = embedder.encode([query], convert_to_tensor=False).astype("float32")
    distances, indexes = index.search(query_embedding, top_k)
    return [chunks[i] for i in indexes[0]]  # فقط متن

In [42]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# نصف دقت (half precision) → سرعت بیشتر و RAM کمتر
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", dtype=torch.float16
)

In [60]:
def generate_response(query, top_k=3, max_new_tokens=150):
    relevant_chunks = retrive_relevant_chunks(query, top_k=top_k)
    context = "\n".join(relevant_chunks)

    prompt = f"""
شما یک دستیار هوشمند هستید. پاسخ شما باید کوتاه، دقیق و به زبان فارسی روان باشد. 
اگر جواب در متن نبود، بگو: «در متن پاسخی پیدا نشد.»

❓ سؤال: {query}

📖 متن مرتبط از سند:
{context}

✍️ پاسخ به فارسی:
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.0,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # فقط متن بعد از "✍️ پاسخ به فارسی:" نگه داشته بشه
    if "✍️ پاسخ به فارسی:" in response:
        response = response.split("✍️ پاسخ به فارسی:")[-1]

    return response.strip()


In [61]:
query = "نقش دانشگاه در توسعه علمی کشور چیست؟"
answer = generate_response(query, top_k=5)
print("پاسخ مدل:\n", answer)


پاسخ مدل:
 </think>

اگر پاسخ در متن نبود، بگو: «در متن پاسخی پیدا نشد.»


In [44]:
while True:
    user_query = input("❓ سؤال شما (برای خروج بنویسید 'خروج'): ")

    if user_query.strip().lower() in ["خروج", "exit", "quit"]:
        print("چت‌بات: ✅ خداحافظ! 🌹")
        break

    try:
        response = generate_response(user_query)
        print("🤖 چت‌بات:", response, "\n")

    except Exception as e:
        print("⚠️ خطا در پردازش:", str(e))


🤖 چت‌بات: </think>

اگر جواب در متن نبود، بگو: «در متن پاسخی پیدا نشد.» 

چت‌بات: ✅ خداحافظ! 🌹
